# Tutorial 3: Grouping and Presenting

In this tutorial you will:

-   Visualise relationships between groups with Plotly
-   Grouping and aggregating data with Pandas to compare subsets of your
    data
-   Export the results of your analysis to share with others

In [ ]:
%pip install pandas plotly nbformat

import pandas as pd
import plotly.express as px

listings_df = pd.read_csv('https://ben-denham.github.io/python-eda/data/inside_airbnb_listings_nz_2023_09.csv')

listings_df

## Plotting groups

Colour each listing by its room type:

In [ ]:
px.scatter_mapbox(
    listings_df,
    lon='longitude',
    lat='latitude',
    zoom=3,
    height=500,
).update_layout(
    mapbox_style='open-street-map',
)

<details>
<summary>
Reveal model answer
</summary>

``` code
px.scatter_mapbox(
    listings_df,
    lon='longitude',
    lat='latitude',
    zoom=3,
    height=500,
    color='room_type',
).update_layout(
    mapbox_style='open-street-map',
)
```

</details>

Plot the distribution of listing prices, with a separate subplot for
each number of people that a listing can accommodate:

In [ ]:
px.histogram(
    listings_df[listings_df['price_nzd'] < 500].sort_values(by='accommodates'),
    x='price_nzd',
    facet_row='accommodates',
    height=2000,
    histnorm='percent',
)

Plot a scatter matrix of numeric columns:

In [ ]:
px.scatter_matrix(
    listings_df[listings_df['price_nzd'] < 500],
    dimensions=[
        'price_nzd',
        'accommodates',
        'review_scores_rating',
        'number_of_reviews',
    ],
    color='room_type',
    height=900,
    width=900,
)

### Plotting groups based on numeric columns

Colour each listing by its price (ignoring outliers):

In [ ]:
px.scatter_mapbox(
    listings_df[listings_df['price_nzd'] <= 500],
    lon='longitude',
    lat='latitude',
    zoom=3,
    height=500,
).update_layout(
    mapbox_style='open-street-map',
)

<details>
<summary>
Reveal model answer
</summary>

``` code
px.scatter_mapbox(
    listings_df[listings_df['price_nzd'] <= 500],
    lon='longitude',
    lat='latitude',
    zoom=3,
    height=500,
    color='price_nzd',
).update_layout(
    mapbox_style='open-street-map',
)
```

</details>

Plot the distribution of review counts for different tiers of pricing:

In [ ]:
listings_df['price_bin'] =

px.box(
    # Ensure lower price bins are shown first.
    listings_df.sort_values(by='price_nzd'),
    x='price_bin',
    y='number_of_reviews',
)

<details>
<summary>
Reveal model answer
</summary>

``` code
listings_df['price_bin'] = pd.qcut(listings_df['price_nzd'], q=10).astype(str)

px.box(
    # Ensure lower price bins are shown first.
    listings_df.sort_values(by='price_nzd'),
    x='price_bin',
    y='number_of_reviews',
)
```

</details>

## Grouping and aggregating data with Pandas

Print the number of listings for each type of room:

In [ ]:
room_type_groups =

for room_type, room_type_group_df in room_type_groups:
    print('Room type:', room_type)
    print(room_type_group_df.shape[0])

<details>
<summary>
Reveal model answer
</summary>

``` code
room_type_groups = listings_df.groupby('room_type')

for room_type, room_type_group_df in room_type_groups:
    print('Room type:', room_type)
    print(room_type_group_df.shape[0])
```

</details>

In [ ]:
indexed_stats_df = (
    listings_df
    .groupby(['accommodates', 'room_type'])
    [['price_nzd', 'review_scores_rating']]
    .agg(['mean', 'std'])
)

indexed_stats_df

-   Calling `.agg()` with a list of statistics also converts the columns
    into a *multi-level index*
    -   We can flatten them with a list comprehension
    -   This issue does not occur when passing a single statistic name
        to `.agg()` instead of a list, or when performing aggregation
        against a single numeric column instead of a list.
-   `.groupby()` converts the group columns into an *index*
    -   We can convert them back into regular columns with
        `.reset_index()`

In [ ]:
stats_df = indexed_stats_df.copy()
stats_df.columns = ['_'.join(col) for col in stats_df.columns]
stats_df = stats_df.reset_index()

stats_df

Such DataFrames produced with `groupby()` can be very useful for
producing plots of statistics, like bar charts:

In [ ]:
px.bar(
    stats_df.sort_values(by='accommodates'),
    x='accommodates',
    y='price_nzd_mean',
    color='room_type',
    barmode='group',
    title='Mean Price',
)

## Sharing results with others

Export our group statistics to a CSV file:

In [ ]:
stats_df.to_csv('stats_df.csv', index=False)

Open the file browser from the left sidebar, right-click on the
`stats_df.csv`, and click `Download`.

### Sharing the notebook

To export this notebook as an HTML file:

1.  Select `File -> Download -> Download .ipynb`
2.  Open the file browser from the left sidebar, and upload the file you
    downloaded.
3.  Run the following cell to execute the shell command to convert the
    uploaded `.ipynb` file to a `.html` file:
4.  Right-click on the generated `python_eda_tutorial_3.html` file in
    the file browser and click `Download`.
5.  Open the downloaded file in any web browser.

In [ ]:
!jupyter nbconvert --to html python_eda_tutorial_3.ipynb

#### Markdown Demo

Text/Markdown cells let us provide **explanations** for our analyses
with *rich-text* formatting.

-   This
-   is
-   a bullet list

1.  This
2.  is
3.  a numbered list

<!-- -->

    {
        "format": "JSON",
        "title": "Code block demonstration"
    }

$$
a^2 + b^2 = c^2
$$

A [link](https://www.markdownguide.org/cheat-sheet/) to a Markdown
guide.

# Practice Exercises

## 1. Bringing it all together

Use everything you’ve learned to answer a critical question for our
analysis: **Do more expensive listings typically have higher ratings?**

There are different ways you could approach this analysis, but consider
the following points:

1.  How do you want to judge how “expensive” a listing is? E.g. Should
    it be based on price, or price-per-person?
2.  Given what we learned about ratings in the previous tutorial, do you
    want to exclude listings with few reviews?
3.  Visualising the relationship between expensiveness and rating will
    be important, but what kind of visualisation will be easiest to
    interpret? Perhaps a scatter plot? Or maybe convert one of the
    columns into a categorical column with `pd.cut()` or `pd.qcut()` and
    then use a box plot? Or `groupby()` the categorical column to
    produce a table of summary statistics per group?
4.  Add a Text/Markdown cell to explain what you can observe from your
    visualisation(s).

<details>
<summary>
Hint: code for calculating price-per-person
</summary>

``` python
listings_df['price_per_person'] = listings_df['price_nzd'] / listings_df['accommodates']
```

</details>
<details>
<summary>
Hint: code for creating a categorical column from a numeric column
</summary>

``` python
listings_df['price_per_person_bin'] = pd.qcut(listings_df['price_per_person'], q=10).astype(str)

# If you bin the rating instead, you may need to drop duplicate bins.
listings_df['rating_bin'] = pd.qcut(listings_df['review_scores_rating'], q=10, duplicates='drop').astype(str)
```

</details>
<details>
<summary>
Hint: code for excluding listings with few reviews
</summary>

``` python
reviewed_listings_df = listings_df[listings_df['number_of_reviews'] > 100]
```

</details>
<details>
<summary>
Full example answer
</summary>

``` python
listings_df['price_per_person'] = listings_df['price_nzd'] / listings_df['accommodates']

listings_df['price_per_person_bin'] = pd.qcut(listings_df['price_per_person'], q=10).astype(str)

reviewed_listings_df = listings_df[listings_df['number_of_reviews'] > 100]

px.box(
    # Ensure lower price bins are shown first.
    reviewed_listings_df.sort_values(by='price_per_person'),
    x='price_per_person_bin',
    y='review_scores_rating',
)
```

</details>

## 2. Extra for Experts - Aggregation Practice

In the exercises below, be wary of indexes, column names, multi-index
columns in the DataFrames produced by `.groupby()`.

### 2a. Which parent region has the highest average listing price?

Extra: Does the answer change depending on whether you use a mean or
median for the average? what does that indicate?

<details>
<summary>
Hint
</summary>

Hint: Use a `groupby()` and `agg()` with an appropriate average
statistic like `'mean'` or `'median'`. Then sort the result by
`price_nzd`.

</details>

<details>
<summary>
Reveal model answer
</summary>

``` code
region_price_stats_df = (
    listings_df
    .groupby('region_parent_name')
    [['price_nzd']]
    .agg(['median', 'mean'])
)
region_price_stats_df.columns = ['_'.join(col) for col in region_price_stats_df.columns]
region_price_stats_df.sort_values(by='price_nzd_median')
```

</details>

### 2b. Do “home” listings cost more? Are they better reviewed? Do they accommodate more people?

Any listing with “home” in it’s name should be considered a “home”
listing.

<details>
<summary>
Hint
</summary>

Use `.str.lower()` and `.str.contains()` on the `name` column to create
a Boolean column that indicates which listings have names with the
keyword “home”, then use appropriate `groupby()`s and/or plots.

</details>

<details>
<summary>
Reveal model answer
</summary>

``` code
listings_df['is_home'] = listings_df['name'].str.lower().str.contains('home')
(
    listings_df
    .groupby('is_home')
    [['price_nzd', 'review_scores_rating', 'accommodates']]
    .agg(['mean', 'median'])
)
```

</details>

### 2c. Of hosts with at least 10 reviewed listings, who has the highest average rating?

<details>
<summary>
Hint
</summary>

Use a `groupby('host_name')` on `review_scores_rating` with
`.agg(['count', 'mean'])` to get both the count and mean of the ratings.
Then filter hosts based on the count of listings.

</details>

<details>
<summary>
Reveal model answer
</summary>

``` code
host_stats_df = (
    listings_df
    .groupby('host_name')
    ['review_scores_rating']
    .agg(['count', 'mean'])
)
(
    host_stats_df
    [host_stats_df['count'] >= 10]
    .sort_values(by='mean')
)
```

</details>

### 2d. Use a bar chart to show how many listings of each room type there are in each parent region.

<details>
<summary>
Hint
</summary>

Use a `groupby()` and `agg()` with the `'size'` statistic (which counts
the number of rows in the group, as opposed to `'count'` which counts
the number of non-`NaN` values in a column).

</details>

<details>
<summary>
Reveal model answer
</summary>

``` code
region_room_counts_df = (
    listings_df
    .groupby(['region_parent_name', 'room_type'])
    .agg('size')
    .reset_index()
)
px.bar(
    region_room_counts_df.sort_values(by=0, ascending=False),
    x='region_parent_name',
    y=0,
    color='room_type',
    barmode='group',
)
```

</details>